In [2]:
import glob
import json
import os

import anthropic

In [3]:
conversations = []

paths = glob.glob('./res/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

target_count = 30
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:           
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

30

In [4]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

In [7]:
print(conversations[-2])

P01: 웹툰이나 만화 자주 보는 편이야?
P02: 나 웹툰은 안 보고 만화는 코난이랑 짱구 슬램덩크 이런 거 좋아해~ 넌?
P03: 난 반대로 만화 안보고 웹툰만 엄청 봐 키키
P03: 요즘에는 시험 때문에 못 보기는 하는데 끝나면 다 몰아볼 예정이야 후욱후욱
P01: 키키 나는 둘 다 안 봄~
P01: 중학교 때는 가끔 만화책 빌려서 봐보긴 했는데 그것도 몇 개 안돼...
P01: 웹툰 재밌어?
P02: 어떤 작가 좋아해?
P02: 특별히 좋아하는 웹툰 있어?
P03: 이동건이라고 유미의 세포들 그린 작가인데 진짜 캐릭처도 귀엽고 감정선이 정말 여자인 건가 싶을 정도로 세심해요!
P03: 재미도 있고 지금도 뭐 신작 나왔더라고!
P01: 아 유미의 세포들! 들어보긴 했어~
P01: 유명하더라 영화인지 드라마로도 나오는 거 같던데? 김고은이 주인공으로
P02: 맞아 드라마 야나? 예고편 같은 거 떴던데~
P03: 웅웅 맞아 맞아 그거 웹툰이 엄청 흥행 해서 드라마화 한다고 하더라...
P03: 근데 개인적으로 웹툰 기반 드라마화 한 거 그렇게 재미는 없을 것 같아서 딱히 기대는 안 하는 중
P01: 맞아 전에 치즈인더트랩도... 별로 였다고 했던 거 같애
P02: 그건 드라마 영화 둘 다 이상해 ㅠㅠ 내용은 괜찮은데~
P03: 맞아 드라마로 했다가 평 안 좋았어서 영화로도 나왔는데 더 망했던 걸로 기억해 키키
P03: 그나마 요번에 한 간 떨어지는 동거 그거는 잘 된 것 같더라고?
P01: 아아 장기용이랑 혜리 나온 거? 뭐 구미호 얘기인 거야...?
P02: 아 진짜? 그것도 앞에만 보다 말았는데~
P03: 웅웅 맞아 맞아 장기용 여기도 나왔네 맞다 키키
P03: 혹시 마블 영화 좋아해?
P01: 보면 재밌는데 나는 막 챙겨보는 편은 아니야! 좋아해?
P02: 안 좋아해~ 다들 재밌다고 해서 맘 잡고 보려고 하는데 눈에 잘 안 들어오더라~ ㅠㅠ
P03: 약간 나랑 언니랑 취향 좀 다른 듯 키키
P03: 나는 마블 영화 엄청 챙겨 보는데 ㅠ
P01: 보면 재

In [24]:
client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

message.usage  # API 호출 비용 대략 50원

Usage(input_tokens=1415, output_tokens=2100)

In [28]:
message.content[0].text.split('\n\n')

["Here's an imagined previous conversation among the three users, following the tone and format of the given conversation:",
 'P02: 안녕~ 오늘 날씨 진짜 좋다~ 너희들 뭐하고 있어?',
 'P03: 나는 지금 집에서 쉬고 있어~ 날씨가 좋아서 창문 열어두고 있는데 기분이 너무 좋아~',
 'P01: 나도 집이야! 오늘 같은 날 밖에 나가고 싶은데 할 일이 너무 많아서 ㅠㅠ',
 'P02: 아 그래? 나는 지금 카페에 와있어~ 커피 마시면서 책 읽고 있는데 너무 좋다~',
 'P03: 와~ 부럽다~ 나도 카페 가고 싶은데... 시험 준비 때문에 집에만 있어야 해서 ㅠㅠ',
 'P01: 맞아 맞아~ 나도 과제랑 시험 준비 때문에 집에만 있어~ 근데 카페에서 공부하면 집중 잘 돼?',
 'P02: 응! 나는 오히려 카페에서 공부하는 게 더 집중이 잘 돼~ 집에 있으면 자꾸 누워있게 되더라고~',
 'P03: 나도 그래~ 집에 있으면 자꾸 침대로 가게 돼서 공부가 안 돼 ㅋㅋㅋ',
 'P01: 아~ 나는 반대야~ 카페에 가면 사람들 구경하느라 집중이 안 돼서 꼭 집에서 공부해~',
 'P02: 그렇구나~ 역시 사람마다 다르네~ 근데 너희 요즘 뭐 재밌는 거 없어? 나 심심해 죽겠어~',
 "P03: 음... 나는 요즘 넷플릭스로 드라마 보고 있어~ '오징어 게임' 봤어?",
 'P01: 아 나도 그거 봤어! 진짜 재밌더라~ 근데 좀 잔인해서 보다가 눈 감은 적도 있어 ㅋㅋㅋ',
 'P02: 나는 아직 못 봤어~ 다들 재밌다고 하던데 나도 한번 봐야겠다~',
 'P03: 응 꼭 봐봐~ 진짜 재밌어! 근데 잔인한 장면이 많아서 밤에 보면 무서울 수도 있어~',
 'P01: 맞아~ 나는 낮에 봤는데도 무서웠어 ㅋㅋㅋ',
 'P02: 그래? 그럼 나는 꼭 낮에 볼게~ 근데 너희 영화는 안 봐?',
 "P03: 나는 영화도 좋아해~ 최근에 본 영화 중에 '듄'이라고 있는데

In [32]:
lst = message.content[0].text.split('\n\n')[1:-1]
for l in lst[:25]:
    print(l)

P02: 안녕~ 오늘 날씨 진짜 좋다~ 너희들 뭐하고 있어?
P03: 나는 지금 집에서 쉬고 있어~ 날씨가 좋아서 창문 열어두고 있는데 기분이 너무 좋아~
P01: 나도 집이야! 오늘 같은 날 밖에 나가고 싶은데 할 일이 너무 많아서 ㅠㅠ
P02: 아 그래? 나는 지금 카페에 와있어~ 커피 마시면서 책 읽고 있는데 너무 좋다~
P03: 와~ 부럽다~ 나도 카페 가고 싶은데... 시험 준비 때문에 집에만 있어야 해서 ㅠㅠ
P01: 맞아 맞아~ 나도 과제랑 시험 준비 때문에 집에만 있어~ 근데 카페에서 공부하면 집중 잘 돼?
P02: 응! 나는 오히려 카페에서 공부하는 게 더 집중이 잘 돼~ 집에 있으면 자꾸 누워있게 되더라고~
P03: 나도 그래~ 집에 있으면 자꾸 침대로 가게 돼서 공부가 안 돼 ㅋㅋㅋ
P01: 아~ 나는 반대야~ 카페에 가면 사람들 구경하느라 집중이 안 돼서 꼭 집에서 공부해~
P02: 그렇구나~ 역시 사람마다 다르네~ 근데 너희 요즘 뭐 재밌는 거 없어? 나 심심해 죽겠어~
P03: 음... 나는 요즘 넷플릭스로 드라마 보고 있어~ '오징어 게임' 봤어?
P01: 아 나도 그거 봤어! 진짜 재밌더라~ 근데 좀 잔인해서 보다가 눈 감은 적도 있어 ㅋㅋㅋ
P02: 나는 아직 못 봤어~ 다들 재밌다고 하던데 나도 한번 봐야겠다~
P03: 응 꼭 봐봐~ 진짜 재밌어! 근데 잔인한 장면이 많아서 밤에 보면 무서울 수도 있어~
P01: 맞아~ 나는 낮에 봤는데도 무서웠어 ㅋㅋㅋ
P02: 그래? 그럼 나는 꼭 낮에 볼게~ 근데 너희 영화는 안 봐?
P03: 나는 영화도 좋아해~ 최근에 본 영화 중에 '듄'이라고 있는데 그거 진짜 좋더라~
P01: 아 나도 그거 보고 싶었는데! 어땠어? 재밌었어?
P03: 응! 진짜 재밌었어~ 영상미도 너무 좋고 스토리도 탄탄해~ 근데 좀 어려울 수 있어~
P02: 나는 그런 SF 영화는 잘 모르겠더라~ 나는 로맨스 코미디가 더 좋아~
P01: 아 그래? 나는 장르 가리지 않고 다 좋아하는 편이야~ 근데 요즘은 시간이 없

In [33]:
for l in lst[25:50]:
    print(l)

P02: 그래그래~ 아 그러고 보니 너희 음악은 뭐 듣고 있어?
P01: 나는 요즘 아이유 노래 많이 듣고 있어~ '라일락' 진짜 좋더라~
P03: 오 나도 그 노래 좋아해! 근데 나는 요즘 방탄소년단 노래를 많이 듣고 있어~
P02: 아 나는 팝송 위주로 듣고 있어~ 요즘 애드 시런 새 앨범이 나왔던데 들어봤어?
P01: 아니 나는 못 들어봤어~ 어때? 좋아?
P02: 응! 진짜 좋아~ 특히 'Shivers'라는 노래가 진짜 중독성 있어~
P03: 오 나도 한번 들어봐야겠다~ 나는 팝송은 잘 모르는데 ㅋㅋㅋ
P01: 나도 그래~ 근데 가끔 팝송 들으면 기분 전환되더라고~
P02: 맞아~ 나는 팝송 들으면서 영어 공부도 하고 있어~
P03: 오 그거 좋은 방법인 것 같아! 나도 한번 해봐야겠다~
P01: 나도! 근데 너희 책은 읽어?
P02: 나는 가끔 읽어~ 요즘은 '데미안'이라는 책 읽고 있어~
P03: 아 그거 유명하지~ 나는 요즘 만화책 많이 읽고 있어~
P01: 오 만화책? 어떤 거 읽고 있어?
P03: 나는 '원피스'랑 '나루토' 이런 거 좋아해~ 너희는?
P02: 나는 만화책은 잘 안 읽어~ 그냥 소설책 위주로 읽는 편이야~
P01: 나도 만화책은 잘 안 읽어~ 근데 웹툰은 가끔 봐~
P03: 아 그래? 어떤 웹툰 봐?
P01: 음... 나는 '여신강림' 이런 거 봐~ 너는?
P03: 나는 '유미의 세포들'이랑 '신의 탑' 이런 거 좋아해~
P02: 아 나는 웹툰도 잘 안 봐~ 그냥 가끔 친구들이 추천해주면 보는 정도?
P01: 웹툰이나 만화 자주 보는 편이야?


In [15]:
print(len(message.content[0].text.split('\n\n')[1]))

2418


In [36]:
lst = message.content[0].text.split('\n\n')[1:-1]
lst

['P02: 안녕~ 오늘 날씨 진짜 좋다~ 너희들 뭐하고 있어?',
 'P03: 나는 지금 집에서 쉬고 있어~ 날씨가 좋아서 창문 열어두고 있는데 기분이 너무 좋아~',
 'P01: 나도 집이야! 오늘 같은 날 밖에 나가고 싶은데 할 일이 너무 많아서 ㅠㅠ',
 'P02: 아 그래? 나는 지금 카페에 와있어~ 커피 마시면서 책 읽고 있는데 너무 좋다~',
 'P03: 와~ 부럽다~ 나도 카페 가고 싶은데... 시험 준비 때문에 집에만 있어야 해서 ㅠㅠ',
 'P01: 맞아 맞아~ 나도 과제랑 시험 준비 때문에 집에만 있어~ 근데 카페에서 공부하면 집중 잘 돼?',
 'P02: 응! 나는 오히려 카페에서 공부하는 게 더 집중이 잘 돼~ 집에 있으면 자꾸 누워있게 되더라고~',
 'P03: 나도 그래~ 집에 있으면 자꾸 침대로 가게 돼서 공부가 안 돼 ㅋㅋㅋ',
 'P01: 아~ 나는 반대야~ 카페에 가면 사람들 구경하느라 집중이 안 돼서 꼭 집에서 공부해~',
 'P02: 그렇구나~ 역시 사람마다 다르네~ 근데 너희 요즘 뭐 재밌는 거 없어? 나 심심해 죽겠어~',
 "P03: 음... 나는 요즘 넷플릭스로 드라마 보고 있어~ '오징어 게임' 봤어?",
 'P01: 아 나도 그거 봤어! 진짜 재밌더라~ 근데 좀 잔인해서 보다가 눈 감은 적도 있어 ㅋㅋㅋ',
 'P02: 나는 아직 못 봤어~ 다들 재밌다고 하던데 나도 한번 봐야겠다~',
 'P03: 응 꼭 봐봐~ 진짜 재밌어! 근데 잔인한 장면이 많아서 밤에 보면 무서울 수도 있어~',
 'P01: 맞아~ 나는 낮에 봤는데도 무서웠어 ㅋㅋㅋ',
 'P02: 그래? 그럼 나는 꼭 낮에 볼게~ 근데 너희 영화는 안 봐?',
 "P03: 나는 영화도 좋아해~ 최근에 본 영화 중에 '듄'이라고 있는데 그거 진짜 좋더라~",
 'P01: 아 나도 그거 보고 싶었는데! 어땠어? 재밌었어?',
 'P03: 응! 진짜 재밌었어~ 영상미도 너무 좋고 스토리도 탄탄해~ 근데 좀 어려울 수 있어~',
 'P02: 나는 그런 SF 영

In [38]:
import pickle

with open('./res/conv_long.pickle', 'wb') as f:
    pickle.dump(lst, f)